## Dynamics for Aircraft Mission Performance

So, what we've demonstrated so far is great for problems where we care about the nitty-gritty of what's happening - we are directly simulating all flight dynamics modes.

For the quadcopter doing a flip, this is a good thing! The vehicle's dynamics have characteristic times that are on the same timescale as the maneuvers of interest, so we should just go ahead and simulate everything.

However, this is overly complex for most aircraft design problems. Most missions of interest are on the order of hours long, while most aircraft flight dynamics modes are on the order of seconds.

We can therefore make some simplifying assumptions:

* The vehicle's short-period modes are negligible, and all modes that don't affect the vehicle energy management (i.e., performance) can be discarded. (On an airplane with classical flight dynamics modes, this would mean that we would ignore all modes other than the phugoid mode.)
* This implies that the angular rates $p$, $q$, and $r$ are zero.
* All lateral modes can be ignored, as we assume that $\beta, \phi, p, r=0$.
* Because the short-period longitudinal mode is negligible and $q=0$, we assume $\theta$ can be prescribed - so $\theta$ is now a control variable, not a state variable.
* Our remaining state variables are $x_e, z_e, u, w$. We can re-parameterize this as $x_e, z_e, V, \gamma$ (where $V$ is the airspeed and $\gamma$ is the flight path angle) to reduce coupling.

Here, we can show what that looks like in practice.

## Optimal Gliding

Let's say you're my Ph.D. advisor, John Hansman, and you're flying over central Massachusetts in your Cessna 152. All of a sudden, you hear your engine start to sputter - oh no.

In [ ]:
thrust = 0

Let's determine:

1. Can he make it back to the airport?
2. What's the most energy-optimal trajectory to fly?